In [1]:
!pip install pyedhrec
!pip install gravis

In [2]:
from pyedhrec import EDHRec
class MyEDHREc(EDHRec):
    def __init__(self):
        super().__init__()
        self._json_page_cards_url = "https://json.edhrec.com/pages/cards/"
    @staticmethod
    def format_card_name(card_name: str) -> str:
        card_name = card_name.replace(" // ", "-")
        return EDHRec.format_card_name(card_name)
    def get_similar_cards(self, cards_name: str):
        formatted_card_name = self.format_card_name(cards_name)
        url = f"{self._json_page_cards_url}{formatted_card_name}.json"
        data = self._get(url)
        return data["similar"]
myedhred = MyEDHREc()
similar_cards = myedhred.get_similar_cards("Dusk // Dawn")
[x["name"] for x in similar_cards]

['Austere Command',
 'Solar Tide',
 'Retribution of the Meek',
 'Slaughter the Strong',
 'Fell the Mighty',
 'Citywide Bust']

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import gravis as gv

class Plotly_Graph_Ploter():
    def __init__(self):
        pass
    def assign_properties(self,g: nx.Graph):
        # Centrality calculation
        node_centralities = nx.in_degree_centrality(g)
        edge_centralities = nx.edge_betweenness_centrality(g)

        # Community detection
        communities = nx.algorithms.community.greedy_modularity_communities(g)

        # Graph properties
        g.graph['node_border_size'] = 1.5
        g.graph['node_border_color'] = 'white'
        g.graph['edge_opacity'] = 0.9

        # Node properties: Size by centrality, shape by size, color by community
        colors = ['red', 'blue', 'green', 'orange', 'pink', 'brown', 'yellow', 'cyan', 'magenta', 'violet']
        for node_id in g.nodes:
            node = g.nodes[node_id]
            node['size'] = 10 + node_centralities[node_id] * 100
            node['shape'] = 'circle'
            for community_counter, community_members in enumerate(communities):
                if node_id in community_members:
                    break
            node['color'] = colors[community_counter % len(colors)]

        # Edge properties: Size by centrality, color by community (within=community color, between=black)
        for edge_id in g.edges:
            edge =  g.edges[edge_id]
            source_node = g.nodes[edge_id[0]]
            target_node = g.nodes[edge_id[1]]
            edge['size'] = edge_centralities[edge_id] * 100
            edge['color'] = source_node['color'] if source_node['color'] == target_node['color'] else 'black'


# Assign node and edge properties
    def plot_graph(self, G: nx.Graph):
        self.assign_properties(G)
        return gv.d3(G, edge_size_data_source='weight', use_edge_size_normalization=True)

In [7]:
from tqdm import tqdm
from IPython.display import HTML, display,DisplayHandle
delimiter= ";"
class SimilarityGraph():
    def __init__(self, graph: nx.Graph = nx.DiGraph()):
        self.graph = graph
        self.ploter = Plotly_Graph_Ploter()
        self.edhrec_asker = MyEDHREc()
        self.count_cards = 0

    def _progress(self,card=None):
        return HTML(f"""
        Current card : {card}<br>
        Number of Cards : {self.count_cards}
        """)
    @staticmethod
    def load_graph(file_path : str):
      return SimilarityGraph(nx.read_adjlist(file_path,delimiter=delimiter))
    def write_graph(self,file_path : str):
      nx.write_adjlist(self.graph,file_path, delimiter=delimiter)
    def add_card(self, card_name: str, depth_similar_cards: int = 1):
        #print(f"Adding {card_name}")
        self.count_add = 0
        self.count_max = sum([6**n for n in range(depth_similar_cards+1)])-1
        display_bar = display(self._progress(),display_id=True)
        self._rec_add_card(card_name,depth_similar_cards,display_bar)

    def _rec_add_card(self, card_name: str, depth_similar_cards: int, display_bar : DisplayHandle):
      similar_cards = self.edhrec_asker.get_similar_cards(card_name)
      self.count_cards +=1
      display_bar.update(self._progress(card_name))
      for card in similar_cards:
        self.graph.add_edge(card_name, card["name"], weight=1)
        if depth_similar_cards > 1 and self.graph.out_degree(card["name"]) == 0:
            self._rec_add_card(card["name"], depth_similar_cards - 1,display_bar)

    def plot_graph(self):
        return self.ploter.plot_graph(self.graph)
simi_graph = SimilarityGraph()
simi_graph.add_card("Counterspell", 3)


In [11]:
simi_graph.plot_graph().export_html("out.html")

In [12]:
simi_graph.write_graph("graph.txt")
!cat graph.txt | head -n 20

#/home/victor/Documents/MTGCardsSimilarity/venv/lib/python3.10/site-packages/ipykernel_launcher.py -f /home/victor/.local/share/jupyter/runtime/kernel-938afbbe-080c-4acd-b647-061843086830.json
# GMT Sun Apr 21 20:34:38 2024
# 
Counterspell;Mana Drain;Spell Pierce;Negate;Dissolve;Disallow;Cancel
Mana Drain;Counterspell;Force of Will;Cryptic Command;Daze;Pact of Negation;Spell Pierce
Force of Will;Force of Negation;Mana Drain;Counterspell;Cryptic Command;Pact of Negation;Daze
Force of Negation
Cryptic Command;Mystic Confluence;Archmage's Charm;Izzet Charm;Ojutai's Command;Silumgar's Command;Kolaghan's Command
Pact of Negation;Force of Will;Force of Negation;Mana Drain;Counterspell;Daze;Spell Pierce
Daze;Force of Will;Force of Negation;Foil;Thwart;Gush;Snap
Mystic Confluence
Archmage's Charm
Izzet Charm
Ojutai's Command
Silumgar's Command
Kolaghan's Command
Foil
Thwart
Gush
Snap


In [141]:
new_graph = SimilarityGraph.load_graph("graph.txt")
list(new_graph.graph.edges)[:10]

[('Counterspell', 'Mana Drain'),
 ('Counterspell', 'Spell Pierce'),
 ('Counterspell', 'Negate'),
 ('Counterspell', 'Dissolve'),
 ('Counterspell', 'Disallow'),
 ('Counterspell', 'Cancel'),
 ('Counterspell', 'Force of Will'),
 ('Counterspell', 'Force of Negation'),
 ('Counterspell', 'Pact of Negation'),
 ('Counterspell', 'Mana Leak')]